# Call Stack

When we compile C code, the resulting x86 code manipulates a special stack called the **call stack**. This is a stack implemented by the processor itself. The purpose of this stack is to preserve the context whenever we make a function call. What does that mean?

Imagine a juggler is performing. She gets stopped mid-routine by some emergency. Somehow, our imaginary juggler is so talented, she remembers exactly where each ball was when she stopped. As soon as the emergency is over, she recovers the routine from the exact moment she stopped. In this analogy, the juggler is our program and the balls are variables. The emergency is a function call.

When we're in the middle of executing a program, variables have been assigned values. The program is not finished using them yet. If we suddenly encounter a function call, we have to drop everything and start executing that function instead of the program we're currently working on. How do we keep track of all of those variable and register values we assigned? How do we remember which line of code we were executing before we got interrupted? That is what we mean by "preserve the context". We have to remember *everything*, so we can pick up *exactly* where we left off.

The technology that solves this problem is the call stack. We push all of the values we want to preserve onto the call stack, including the address of the line of code we want to come back to. We call this the **return address**. When we're done executing the function, we can pop all these values off the stack, restore them, and continue execution.

In this notebook, we will explore the x86 call stack in detail.

## x86 Call Stack

The x86 call stack lives in memory. The assembly we will see is all 32-bits. There are 8 bits in a byte and $\frac{32}{8}=4$, so the memory addresses, registers, and most of the values we'll deal with are 4-bytes long. This is sometimes called the **word size** of the processor. For 32-bit addresses, smallest memory address is 0x00000000 and the largest address is 0xFFFFFFFF. Memory for a process is laid out something like this:

| Memory addresses | Contents                                        | 
|------------------|-------------------------------------------------|
| 0x00000000       | code for our program (the machine instructions) |
| ...              | data for our program (e.g. string literals)     |
| ...              | heap (we might discuss this later)              |
| ...              | ...                                             |
| ...              | ...                                             |
| 0xFFFFFFFF       | stack                                           |

The bottom of the x86 call stack is in high memory, close to 0xFFFFFFFF. As we push items onto the stack, the top of the stack grows up into lower memory addresses, toward 0x00000000.

The memory address of the top of the stack is stored in register ESP, called the **stack pointer**. The x86 **push instruction** subtracts 4 from the value of ESP, then stores data in the memory address in ESP. For example, if ESP stored the value 0xFFFFFF48, a push instruction would update ESP to 0xFFFFFF44 and the value pushed would be stored in memory address 0xFFFFFF44. The **pop instruction** does the opposite. It retreives the values stored at ESP, then adds 4 to ESP.

Why add and subtract 4? Every value stored on the stack is 4 bytes long, so we need to move in increments of 4 bytes. The word size is 4 bytes.

We group data in the x86 call stack into frames. A **stack frame** refers to *all* the data stored on the stack for a particular function. Every time we call a function, we push a new stack frame onto the call stack. Every time we return from a function, we pop its entire stack frame off of the call stack. A stack frame looks something like this:

| Contents                    | Registers        |
|-----------------------------|----------------- |
| Local variables             | $\leftarrow$ ESP |
| ...                         | ...              |
| Previous frame pointer      | $\leftarrow$ EBP | 
| Return address of caller    | ...              |
| Function arguments          | ...              |
| ...                         | ...              |

We'll learn more about the contents of a stack frame as we go. For now, know that there's another special register called EBP that keeps track of the bottom of the current stack frame. All of the memory between EBP and ESP is in the current stack frame.

Whew! That's a *lot* to take in! Let's see the call stack in action with an example C program. If you look in the file "call_stack.c", you'll find a simple C program. The main program just declares 3 local variables, calls the function "foo" twice, and then prints a result.

Once again, this is C code. **This won't run in Jupyter**.

In [ ]:
#include <stdio.h>

int foo(int x) {
   return x-0xFACE;

}

int main(){
        int a=0xABADD00D;
        int b=0xC0DEBABE;
        int c=0;
        c=foo(a);
        c=c+foo(a);
        printf("Result: %x\n",c);
        return 0;
}

## Make

To compile code in this assignment, **don't use gcc directly!** In other words, don't try to compile the code yourself. Instead, I've included a "Makefile" which will automatically build the programs for you. If you run the command "make", *all* of the C programs for this assignment will automatically compile.

You should run "make" now. It will produce an executable named "call_stack". Let's explore this executable in gdb. First, we need to do a little setup. In your home directory, we need to create a file named ".gdbinit" and add the line "set disassembly-flavor intel" to it. You can do that yourself or you can do it by running this command:

echo "set disassembly-flavor intel" > ~/.gdbinit

*Now* we're ready for gdb. Run the command "gdb call_stack". If you try to view the source code in gdb using "list main" like we did in the previous gdb notebook, you'll see that this command fails. "call_stack" wasn't compiled to be friendly to the debugger. It was compiled so we can explore the call stack. All of the other gdb commands you learned last week should still work, though.

## Call Stack in GDB

Let's run a scary gdb command. Don't panic when you see the output. Type "disas main" to view the assembly code for call_stack.c. Let's break down what we're looking at.

Each line represents one machine instruction. The leftmost column is the memory address of each instruction. The program is stored in RAM. In that leftmost column, we're seeing the location in RAM each instruction inhabits. The middle column is the instruction's name. That tells us what operation this instruction is supposed to do. 

| Memory address    | Instruction name | Operands |
|-------------------|-------|-------------------|
|0x080488b2 <+0>:   |  lea  |  ecx,\[esp+0x4\] |
|   0x080488b6 <+4>: |    and  |   esp,0xfffffff0 |
|  0x080488b9 <+7>:  |   push  | DWORD PTR \[ecx-0x4\] |
| 0x080488bc <+10>:  |  push  | ebp |

For example, "and" means performing a bitwise and operation (like the & operator in Python). "Push" means push a value onto the call stack. The last column contains the operands. This describes the data we are manipulating with the instruction. For example, "and esp,0xfffffff0" means compute the bitwise and of the value in register esp and 0xfffffff0 and store the result back in esp. The instruction "push ebp" means push the value stored in register ebp onto the stack. The command "push \[ecx-0x4\]" means get the value in register ecx and subtract for. The result is a memory address. Retrieve the data there push it onto the stack. Square brackets in x86 always mean "access memory". 

We don't need to understand what all of the x86 assembly code is doing. Most computer science students wouldn't even be able to explain what every line of this assembly code is doing. We're here to learn about the call stack. Let's focus on that.

## Local Variables
First, let's explore how local variables are handled on the call stack. All local variables are stored on the stack. Whenever we declare a local variable in C, corresponding x86 code allocates space for it on the stack. To do that, we need to move the stack pointer up. Remember, we move the stack pointer *higher* by *subtracting* from ESP. The sub instruction at memory address 0x080488c0 allocates 20 (0x14 is hexadecimal) bytes on the stack. 

sub    esp,0x14

In gdb, set a breakpoint at this instruction like this:

break \*0x080488c0

Run the program (the "run" command) until you hit this breakpoint. Now, if you type the command "info registers", you should see the values in all of the registers. Take note of the value stored in ESP. That's address of the top of the stack.

We can execute just one instruction with the command "si" (step instruction). Execute this one instruction and look at the registers again. You should see that the ESP register was decremented by 20 (0x16).

16 bytes is more than enough space to store the 3 integers "a", "b", and "c" The code starting at address 0x080488c3 initializes the values of the variables "a", "b", and "c". There is a direct correspondence between this assembly and the original C code.

| C instruction      | Assembly instruction          |
|--------------------|-------------------------------|
|int a=0xABADD00D;   | mov   \[ebp-0xc\],0xabadd00d  |
|int b=0xC0DEBABE;   | mov   \[ebp-0x10\],0xc0debabe |
|int c=0;            | mov   \[ebp-0x14\],0x0        |

If we execute these three instructions, (run "si" three more times) we can now view these variables on the stack. In gdb, we can view data stored at a memory address with the "x/wx" (e**x**amine **w**ord in **h**exadecimal) command. For example, "x/wx 0xffffd3b8" shows us the data stored in memory adress 0xffffd3b8. Let's look at the data stored in memory address ebp-0xc. We can do this with this command "x/wx \\$ebp-0xc".

Local variables are stored on the stack relative to the base pointer. In our example, the compiler has decided that variable "a" will live in memory address ebp-0xc, variable "b" will live at memory address ebp-0x10, and variable "c" will live at memory address ebp-0x14. Using gdb, we can peek at the value of each variable. "x/wx \\$ebp-0xc" shows us "a". "x/wx \\$ebp-0x10" displays the vaue of "b". "x/wx \\$ebp-0x14" displays the value of "c". 

If we see data in any of those locations being manipulated, we know the assembly code is manipulating the value of that variable. For example, the very next instruction at memory address 0x080488d8 pushes the value of the variable "a" onto the stack:

push    \[ebp-0xc\]

Run this command (the "si" command again.) Let's see what's on top of the stack. We can do that with x/wx \\$esp. I doubt you'll be surprised by the result.

Okay. But why are we pushing the value of "a" onto the stack? That's a little strange.

## Calling a Function

Just before we call a function, we push all the arguments to the function onto the stack. That's the procedure we follow to give the arguments to the function we're calling. So, to answer the question from the previous question, we're pushing "a" onto the stack because it's an argument to "foo". The C code includes this line:

c=foo(a);

We're setting up this function call by pushing the argument "a" onto the stack. The next instruction calls foo:

call   0x80488a5 \< foo\>
    
The call instruction does two things. First, it pushes the address of the next instruction onto the stack. That's so we can remember where to go back to when we're done with foo. We call this the **return address** In this case, the value 0x080488e0 gets pushed onto the stack. Then, we jump to memory address 0x80488a5, the start of the function foo.

Your debugger should be waiting to execute this call instruction (if you type disas main, you should see a little arrow next to this instruction.) Let's execute it (the "si" command again). Let's see what's on top of the stack (the "x/wx \\$esp" command). The value there should be the address of the next instruction in main after the call instruction.

0x080488e0 \<+46\>:    add    esp,0x4

Why do we push the address of the next instruction onto the stack? We're about to jump away to a strange new subprogram. We need to remember how to get back to this location and continue executing. This is a reminder to return to what we were doing before we got interrupted. Again, this is called the **return address**. I mentioned it twice, because it is *very* important to us.

## Entering a Function

When we enter a function, we almost always execute a common pattern of two instructions. If you look at the assembly code for foo, you'll see the following two instructions:

push   ebp  
mov    ebp,esp

These instructions set up the new stack frame. They're called the "function prologue". First, we need to preserve the previous value of EBP for the previous stack frame. We're about to overwrite EBP with a new value. That's why we push the old value onto the stack. Second, we need to update EBP so it points to the beginning of the new stack frame. Where is that? Wherever the current top of the stack is. How do we know where the top of the stack is? ESP stores that value. That's why we move the contents of ESP into EBP.

Step over these two instructions ("si" twice). Now that the new stack frame is set up, we're ready to execute the body of the "foo" function.

## Returning from a Function
"foo" doesn't do much. It just computes an expression and returns a value.

In [ ]:
int foo(x){
   return x-0xFACE;
}

By convention, we return values in the EAX register. When this function is finished, we want the result of x-0xFACE to go in EAX. In other words, we're going to build our return value in EAX.

First, we need to get the value of "x". Where is it? If you recall, main pushed the argument onto the stack before calling "foo". That means it's in the previous stack frame, or "below" the base pointer on the stack. You can view 16 values after EBP on the stack with the command "x/16wx $ebp". "x" should have the value 0xABADD00D. Can you see it? It's two words (8 bytes) past the base pointer. That's why the next instruction is:

mov    eax,DWORD PTR \[ebp+0x8\]

If you execute this instruction ("si") eax has the value of "x" in it. Check with "info registers" to confirm. Next, we subtract 0xFACE.

sub    eax,0xface

Stepping over that instruction ("si") will leave us with the result in EAX. If you check "info registers" again, you should see that 0xABADD00D-0xFACE=0xabacd53f is now stored in EAX. The function's work is done. Time to return to main.

To return, we need to break down the stack frame. We saw the "function prologue" earlier. This is the "function epilogue". First, we pop the old value of EBP off of the stack and store it back in EBP. This restores EBP to its previous value.

pop    ebp

Step over that instruction ("si"). What value is on top of the stack ("x/wx $esp") right now? The next instruction (ret) will pop that value off the stack and jump to that address. Remember when main pushed the return address onto the stack? We're popping it back off now. If you look in main ("disas main") you should see that the value on top of the stack right now is the address of the next instruction in "main" after the call to "foo". If you step again ("si"), you'll see that we jump to that instruction in main and continue executing main.

## Summary

So there you have it. We just watched in detail how the x86 call stack is used through the entire process of calling a function. We saw how each of these components of a stack frame is built and used:

| Contents                    | Registers        |
|-----------------------------|----------------- |
| Local variables             | $\leftarrow$ ESP |
| ...                         | ...              |
| Previous frame pointer      | $\leftarrow$ EBP | 
| Return address of caller    | ...              |
| Function arguments          | ...              |
| ...                         | ...              |


## Exercises
The exercises in this notebook continue where we left off with "call_stack.c". You should continue to explore it with gdb.

1) Set a breakpoint for address 0x080488e3 and run until you hit this breakpoint. You should be waiting to execute this instruction:

mov    DWORD PTR \[ebp-0x14\],eax

What is the role of this instruction? What variable is at memory address ebp-0x14? Why are we storing the contents of eax there? After you execute this instruction, what value is stored in ebp-0x14? Why?

2) The next instruction is:

push   DWORD PTR \[ebp-0xc\]

What variable is at memory address ebp-0xc? What value is stored there? Why are we pushing it onto the stack?

**HINT:** The next instruction is a call instruction. What do we push onto the stack before a call instruction?

3) Set a breakpoint for the last instruction of "foo" (0x080488b1). Run until you hit this breakpoint. What value is on top of the stack? What does this value represent? How did it get there and what will we do with it?

NOTE: This should be the **second** time "foo" is called. If you restarted the debugger, make sure go to the second time with the "continue" command.

4) Set a breakpoint for the call to "printf" (0x080488ff) and run until you hit this breakpoint. The two instructions before this call are:

push   DWORD PTR \[ebp-0x14\]  
push   0x80abdc8

What are these values? Why are we pushing them onto the stack?

**HINT 1:** 0x80abdc8 is a memory address.  
**HINT 2:** The gdb command "x/s" will display strings instead of hex.